In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from itertools import product
import warnings
warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8")

import sys
sys.path.append('/Users/markwindsor/Desktop/atlas_trade')
from src.utils.technical_indicators import *
# Use your absolute path to our top 500 coins csv
data_path = '/Users/markwindsor/Desktop/atlas_trade/src/data/ETHUSDT_30days.csv'

data = pd.read_csv(data_path)

In [46]:
class MeanRevertingCrypto():

    def __init__(self,  symbol, start_date, trading_costs):
        self.symbol = symbol
        self.start_date = start_date
        self.trading_costs = trading_costs
        self.results = None
        self.data_df = self.get_data()
        
    def __repr__(self):
        return "Mean Reverting Crypto (symbol = {}, start = {})".format(self.symbol, self.start_date)
    
    def get_data(self):
        data['returns'] = np.log(data.close / data.close.shift(1))
        data.set_index('time', inplace=True)
        return data
        
    
    def prepare_data(self):

        data = self.data_df.copy()

        # ******************** define your strategy here ************************
        data = relative_strength_index(data)
        data = bollinger_bands(data)

        data.dropna(inplace=True)

        # Buy Conditions
        bcond1 = data['rsi'].iloc[-1] < 30
        bcond2 = data['close'].iloc[-1] <= data['lower_band'].iloc[-1]

        buy_cond = bcond1 & bcond2

        # Sell Conditions
        scond1 = data['rsi'].iloc[-1] > 70
        scond2 = data['close'].iloc[-1] >= data['upper_band'].iloc[-1]

        sell_cond = scond1 & scond2

        data["position"] = 0
        data.loc[buy_cond, "position"] = 1
        data.loc[sell_cond, "position"] = -1

        data.to_csv('results.csv', index=False)

        self.results = data.copy()

    def run_backtest(self):

        data = self.results.copy()
        data['strategy'] = data.position.diff().fillna(0).abs()
        data['trades'] = data.position.diff().fillna(0)
        data["strategy"] = data["position"].shift(1) * data["returns"]
        data["trades"] = data.position.diff().fillna(0).abs()
        data.strategy = data.strategy + data.trades * self.trading_costs

        self.results = data

    def test_strategy(self):

        self.prepare_data()
        self.run_backtest()
        
        data = self.results.copy()
        data["creturns"] = data["returns"].cumsum().apply(np.exp)
        data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
        self.results = data
        
        self.print_performance()

    def plot_results(self, leverage = False): #Adj!
        '''  Plots the cumulative performance of the trading strategy compared to buy-and-hold.
        '''
        if self.results is None:
            print("Run test_strategy() first.")
        elif leverage: 
            title = "{} | Trading Costs = {} | Leverage = {}".format(self.symbol, self.trading_costs, self.leverage)
            self.results[["creturns", "cstrategy", "cstrategy_levered"]].plot(title=title, figsize=(12, 8))
        else:
            title = "{} | Trading Costs = {}".format(self.symbol, self.trading_costs)
            self.results[["creturns", "cstrategy"]].plot(title=title, figsize=(12, 8))

    def print_performance(self, leverage = False): # Adj
        ''' Calculates and prints various Performance Metrics.
        '''
        
        data = self.results.copy()
        
        if leverage: # NEW!
            to_analyze = np.log(data.strategy_levered.add(1))
        else: 
            to_analyze = data.strategy
            
            
        strategy_multiple = round(self.calculate_multiple(to_analyze), 6)


        print(100 * "=")
        print("Pure Technical| INSTRUMENT = {} |".format(self.symbol))
        print(100 * "-")
        print("PERFORMANCE MEASURES:")
        print("\n")
        print("Multiple (Strategy):         {}".format(strategy_multiple))

        print(38 * "-")

        
        print(100 * "=")

    def calculate_multiple(self, series):
        return np.exp(series.sum())
    
    # def calculate_cagr(self, series):
    #     return np.exp(series.sum())**(1/((series.index[-1] - series.index[0]).days / 365.25)) - 1
    
    # def calculate_annualized_mean(self, series):
    #     return series.mean() * self.tp_year
    
    # def calculate_annualized_std(self, series):
    #     return series.std() * np.sqrt(self.tp_year)
    
    # def calculate_sharpe(self, series):
    #     if series.std() == 0:
    #         return np.nan
    #     else:
    #         return self.calculate_cagr(series) / self.calculate_annualized_std(series)



In [47]:
symbol = "ETHUSDT"
start_date = '2023-08-16'
tc = -0.0005


tester = MeanRevertingCrypto( symbol = symbol,
                              start_date = start_date,  trading_costs = tc)

tester.prepare_data()
tester.run_backtest()
tester.test_strategy()

Pure Technical| INSTRUMENT = ETHUSDT |
----------------------------------------------------------------------------------------------------
PERFORMANCE MEASURES:


Multiple (Strategy):         1.0
--------------------------------------
